# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [8]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [9]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [11]:
import pandas as pd

# Table names and definitions
tables = ["users", "orders", "products"]
definitions = [
    "Stores user information such as name, email, and age.",
    "Stores order details including user ID, product ID, and order date.",
    "Contains product details like name, price, and stock."
]

# Create DataFrame
data = {'table': tables, 'definition': definitions}
df = pd.DataFrame(data)

# Print DataFrame
print(df)


      table                                         definition
0     users  Stores user information such as name, email, a...
1    orders  Stores order details including user ID, produc...
2  products  Contains product details like name, price, and...


In [13]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [14]:
print(text_tables)

users: Stores user information such as name, email, and age.
orders: Stores order details including user ID, product ID, and order date.
products: Contains product details like name, price, and stock.


In [15]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [17]:
# Creating the prompt with user question and table definitions
pqt1 = prompt_question_tables.format(tables=text_tables, question="ENTER YOUR QUERY HERE")


In [18]:
print(return_OAI(pqt1))

{
    "tables": ["users", "orders", "products"]
}


In [23]:
pqt3 = prompt_question_tables.format(tables=text_tables, 
                                     question="What is the total number of users?")


In [24]:
print(return_OAI(pqt3))

```json
{
    "tables": ["users"]
}
```


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [25]:
pqt4 = prompt_question_tables.format(tables=text_tables, 
                                     question="What products have been ordered by users over the age of 30?")
print(return_OAI(pqt4))

```json
{
    "tables": ["users", "orders", "products"]
}
```


In [26]:
pqt5 = prompt_question_tables.format(tables=text_tables, 
                                     question="What is the total sales for each product?")
print(return_OAI(pqt5))

{
    "tables": ["orders", "products"]
}


In [27]:
pqt6 = prompt_question_tables.format(tables=text_tables, 
                                     question="What are the names of users who have ordered a specific product?")
print(return_OAI(pqt6))

```json
{
    "tables": ["users", "orders", "products"]
}
```


Findings Report:
1. Results:
Question 1: "What products have been ordered by users over the age of 30?"

Required tables: users, orders, products.

Result: The tables were correctly identified, as the model needs all three tables to answer the question.

Question 2: "What is the total sales for each product?"

Required tables: orders, products.

Result: The tables were correctly identified, as the model needs the orders and products tables to calculate total sales.

Question 3: "What are the names of users who have ordered a specific product?"

Required tables: users, orders.

Result: The tables were correctly identified, as the model needs the users and orders tables to link users to the products they ordered.

2. Variations That Did Not Work Well:
Sometimes, when the question is ambiguous or requires additional information, the model may provide an inaccurate answer. For example, if the question involves data not present in the specified tables, the model might "hallucinate" an answer instead of indicating that the data is unavailable.

3. What I Learned:
Model Accuracy: The model performs well when the questions are clear and match the available data in the tables.

Ambiguity in Questions: If the question is ambiguous or requires additional information, the model may provide an inaccurate answer or attempt to guess the answer.

Importance of Table Definitions: Clearly defining the tables helps the model understand the relationships between them and accurately determine which tables are needed.

3. Conclusion:
Through this experiment, I learned that the model performs well when the questions are clear and specific, and that accurately defining the tables helps improve the accuracy of the answers. However, we must be cautious with ambiguous questions that may lead to inaccurate responses.

